### Computational Guided Inquiry for Modeling Earth's Climate (Neshyba, 2025)

# Cambio 2.0

## Equations of motion of Cambio2.0
The equations of motion of Cambio2.0 are almost the same as Cambio1.0. The sole difference is that Cambio2.0 takes into account the *Henry effect*, by which the temperature dependence of the solubility of carbon dioxide in the ocean is taken into account.

$$
F_{land->atm} =  k_{la} \ \ \ (1) 
$$

$$
F_{atm->land} = k_{al0} +  k_{al1} \times [C_{atm}] \ \ \ (2)
$$

$$
F_{ocean->atm} = k_{oa} \times (1+DC\times T_{anomaly}) [C_{ocean}] \ \ \ (3)
$$

$$
F_{atm->ocean} = k_{ao} [C_{atm}] \ \ \ (4)
$$

$$
F_{human->atm} = \epsilon(t) \ \ \ (5)
$$

This innovation is a first step toward incorporating feedbacks into Cambio (of which more to come later).

## The *meclib* library and other resources
We've packed a lot of functions that you're familiar with into a resources library, *MEclib*. Here's a partial list of them:

- Diagnose_OceanSurfacepH  
- Diagnose_T_anomaly  
- Diagnose_actual_temperature  
- Diagnose_degreesF  

We're also introducing a new dictionary to contain variables that define the current state of the climate (called *ClimateState*), and new functions that solve the Cambio equations of motion (called *PropagateClimateState_Cambio1* and *PropagateClimateState_Cambio2*).

## Uploading your climate emissions scenario
As before, you'll need to upload a climate emissions scenario file (like "Peaks_in_2040.pkl", or whichever one you prefer) to the current folder.

## Learning goals
1. I can make effective use of Python's "help" command.
1. I can explain how the *Henry effect* plays a role in these equations, and in their results.
1. I can make lists of Python dictionaries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meclib.cl as cl
from copy import copy as makeacopy

In [ ]:
%matplotlib notebook

### Python's "help" command
The cell below shows how to get details about functions. Try it!

In [ ]:
help(np.log10)
help(cl.Diagnose_OceanSurfacepH)

### Your turn
Use Python's "help" command to get help on a couple of functions in the cl library (check out the list in the Introduction).

In [ ]:
# your code here 


### Loading the scheduled flow
In the cell below, do what you did before in DiagnosticFunctions -- i.e., load in the scheduled flows file you have uploaded. It'll be most convenient if you use the following naming convention: 

    time, eps, epsdictionary_fromfile = cl.LoadMyScenario(filename)

(but of course supplying the name of your own scheduled flows file).

In [ ]:
# your code here 


### Creating a dictionary for climate parameters
In the cell below, we use the CreateClimateParams function to create a dictionary of climate parameters.

In [ ]:
ClimateParams = cl.CreateClimateParams(epsdictionary_fromfile)
display(ClimateParams)

### Creating a climate state

In [ ]:
ClimateState = cl.CreateClimateState(ClimateParams)
display(ClimateState)

### Making a list of climate states
Eventually, we're going to want to make list of climate states. In the next cell, we practice that process by making a very simple and boring list -- two identical preindustrial climate states. 

You might notice that the syntax for this is different from how we appended to make lists before. That's because here we're creating a regular Python list, whereas before we were creating a list of numpy arrays.

In [ ]:
ClimateState_list = []
ClimateState_list.append(ClimateState)
ClimateState_list.append(ClimateState)
display(ClimateState_list)

### Extracting a time series from a climate list
The "display" capability we just used is handy to get a view of the contents of a list, but sometimes we're more interested in particular items in the list. The function CollectClimateTimeSeries does just that. In the cell below, you can see how it works. Of course there are just two values, but when you use CollectClimateTimeSeries later, there will be hundreds of values.

In [ ]:
time_array = cl.CollectClimateTimeSeries(ClimateState_list,'time')
display(time_array)

### Your turn
Collect and display all the atmospheric reservoir carbon amounts (C_atm) in ClimateState_list.

In [ ]:
# your code here 


### Duplicating the Cambio1.0 algorithm 
The cell below duplicates Cambio1.0's algorithm for propagating the climate state forward, but implemented as a Python function. This is all done for you, but it's worth taking a few minutes to inspect the code so you understand how it works.

One item in particular to note: with this architecture, the array of climate states stored as a list is one shorter than the original. That's why the time plots have "time_array" on the x-axis (instead of "time").

In [ ]:
def PropagateCS_Cambio1(previousClimateState, ClimateParams, dt, F_ha):
    """Propagates the state of the climate, with a specified anthropogenic carbon flux"""
    """Returns a new climate state"""

    # Extract constants from ClimateParams
    k_la = ClimateParams['k_la']
    k_al0 = ClimateParams['k_al0']
    k_al1 = ClimateParams['k_al1']
    k_oa = ClimateParams['k_oa']
    k_ao = ClimateParams['k_ao']
    DC = ClimateParams['DC']
    preindustrial_albedo = ClimateParams['preindustrial albedo']
    fractional_albedo_floor = ClimateParams['fractional_albedo_floor']
    albedo_Tstar = ClimateParams['albedo_Tstar']
    albedo_delta_T = ClimateParams['albedo_delta_T']
    k_al1_Tstar = ClimateParams['k_al1_Tstar']
    k_al1_deltaT = ClimateParams['k_al1_deltaT']
    fractional_k_al1_floor = ClimateParams['fractional_k_al1_floor']
    
    # Extract concentrations, albedo, etc, from the previous climate state
    C_atm = previousClimateState['C_atm']
    C_ocean = previousClimateState['C_ocean']
    albedo = previousClimateState['albedo']
    time = previousClimateState['time']
    
    # Get the temperature implied by the carbon in the atmosphere, and ocean pH
    T_anomaly = cl.Diagnose_T_anomaly(C_atm, preindustrial_albedo, ClimateParams)
    actual_temperature = cl.Diagnose_actual_temperature(T_anomaly)
    OceanSurfacepH = cl.Diagnose_OceanSurfacepH(C_atm,ClimateParams)
    
    # Get new fluxes (including the effect of temperature anomaly on the ocean-to-atmosphere flux)
    F_la = k_la    
    F_al = k_al0 + k_al1*C_atm
    F_oa = k_oa*C_ocean    
    F_ao = k_ao*C_atm

    # Get new concentrations of carbon that depend on the fluxes
    C_atm += (F_la + F_oa - F_ao - F_al + F_ha)*dt
    C_ocean += (F_ao - F_oa)*dt
    time += dt
    
    # Create a new climate state with these updates
    ClimateState = makeacopy(previousClimateState)
    ClimateState['C_atm'] = C_atm
    ClimateState['C_ocean'] = C_ocean
    ClimateState['F_al'] = F_al
    ClimateState['F_la'] = F_la
    ClimateState['F_ao'] = F_ao
    ClimateState['F_oa'] = F_oa
    ClimateState['F_ha'] = F_ha
    ClimateState['F_ocean_net'] = F_oa-F_ao
    ClimateState['F_land_net'] = F_la-F_al
    ClimateState['time'] = time
    ClimateState['T_anomaly'] = T_anomaly
    ClimateState['actual temperature'] = actual_temperature
    ClimateState['OceanSurfacepH'] = OceanSurfacepH
    ClimateState['albedo'] = albedo

    # Return the new climate state
    return ClimateState

# Run Cambio1.0
CS_Cambio1_list = cl.run_Cambio(PropagateCS_Cambio1, ClimateParams, time, eps)

# Choose items to plot
items_to_plot = [['C_atm','C_ocean'],['F_ha','F_ocean_net','F_land_net'],'T_anomaly','albedo']

# Plot those items
cl.CS_list_plots(CS_Cambio1_list,'Cambio1',items_to_plot)

### Upgrading to Cambio2.0
Below, the idea is to tweak what we just did such that we include Henry's effect (Eq. 3 in the Introduction). It's highly recommended that you make these tweaks one at a time, with plenty of testing after each one.

Here are the changes we'll need in the cell below:
1. Don't forget to name your function PropagateClimateState_Cambio2 (not PropagateClimateState_Cambio1), and to pass it to cl.run_Cambio.
1. Inside PropagateClimateState_Cambio2, use the temperature anomaly to implement Henry's effect -- as represented by Eq. 3 in the Introduction.
1. In addition to plotting the timelines you made for Cambio1.0, modify the variable *items_to_plot* to include 'OceanSurfacepH'.

In [ ]:
# your code here 


### Pause for analysis
1. Comparing Cambio 1.0 to 2.0 (i.e., the effect of incorporating the Henry's Law effect of temperature on the oceans), how much warmer is the peak temperature according to your Cambio 2.0 results vs 1.0? 
1. Focusing on just Cambio 2.0, how much of a time lag is there between peak emissions and peak temperature And what's the most acidic pH?

YOUR ANSWER HERE

### Validating and finishing up
Assuming all this has gone smoothly, don't forget to do a Kernel/Restart & Run All, run the whole notebook, and make sure there aren't any errors.